In [3]:
!pip install konlpy
!pip install gensim
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 46.6 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import os
import pandas as pd
import csv
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from konlpy.tag import Okt
from tqdm import tqdm
from gensim.models import Word2Vec
import time
import random
import numpy as np

# 랜덤 시드 설정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# 텍스트 전처리 함수 정의
def preprocess_text(text, okt, stop_words):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    tokens = okt.pos(text) # 토큰화 후 형태소 단위로 분석, [(단어, 품사)] 리턴
    tokens = [word for word, pos in tokens if pos in ['Josa', 'Noun', 'Verb']] # 명사, 동사, 조사 이외의 단어들은 제거
    tokens = [word for word in tokens if word not in stop_words] # 불용어 제거
    return tokens

# CSV 파일 읽어서 저장 기사 Content 튜플
def read_csv_file(file_path):
    titles = []
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            titles.append(row['Content'])
    return titles

# 전처리된 제목 리스트 생성 함수 정의, tqdm으로 시각화해서 token화된 list 반환
def preprocess_titles(titles, okt, stop_words):
    return [preprocess_text(title, okt, stop_words) for title in tqdm(titles, desc="Processing titles")]

# Word2Vec 모델 학습 및 저장 함수 정의
def train_and_save_word2vec_model(processed_titles, model_path, seed=SEED):
    # 디렉터리가 존재하지 않으면 생성
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    model = Word2Vec(sentences=processed_titles, vector_size=100, window=5, min_count=1, workers=4, sg=1, seed=seed)
    model.save(model_path)
    return model

# 저장된 Word2Vec 모델 불러오기 함수 정의
def load_word2vec_model(model_path):
    return Word2Vec.load(model_path)

# TF-IDF 벡터화 및 희소 행렬 생성 함수 정의
def calculate_tfidf(processed_titles_str):
    # 파라메터 조정 가능
    # ngram_range=(2, 3) : 토큰을 묶는 방식 파라메터 default 1:1
    # min_df=2 : 해당 값보다 낮은 빈도수의 토큰(용어)은 무시한다.
    # max_df=0.7 : 너무 자주 등장하는 토큰은 무시한다.
    # stop_words=[] : 불용어 리스트, 해당 리스트에 있는 단어는 무시
    # analyzer='char' : char, char_wb 있음
    tfidfvect = TfidfVectorizer()
    dtm = tfidfvect.fit_transform(processed_titles_str) # 벡터화해서 Tf-idf 계산
    feature_names = tfidfvect.get_feature_names_out() # 벡터화된 단어 목록 반환
    # dense = dtm.todense() # 희소행렬반환
    # print(dense)
    # denselist = dense.tolist() # 밀집행렬로 반환
    return feature_names, dtm

# TF-IDF 결과 출력 함수 정의 -> 문서에 tf-idf 결과 확인용
def print_tfidf_results(feature_names, dtm, num_docs=10):
    for i in range(num_docs):
        print(f"문서 {i+1}의 TF-IDF 결과:")
        doc = dtm.getrow(i).tocoo()  # 희소행렬의 i번째 행을 COO 포맷으로 변환
        sorted_phrase_scores = sorted(zip(doc.col, doc.data), key=lambda t: t[1], reverse=True)
        for phrase, score in sorted_phrase_scores:
            print(f"  {feature_names[phrase]}: {score}")
        print("\n")

# 입력 단어에 대한 유사 단어 찾기 및 TF-IDF 값 계산 함수 정의
def find_similar_words(word2vec_model, input_word, feature_names, dtm):
    if input_word not in word2vec_model.wv:
        print(f"단어 '{input_word}'가 Word2Vec 모델의 어휘에 없습니다.")
        return {}
    similar_words = word2vec_model.wv.most_similar(input_word, topn=10)
    similar_words_tf_idf = {}
    for word, similarity in similar_words:
        if word in feature_names:
            index = feature_names.tolist().index(word) # 해당 단어의 열 위치
            # 희소행렬에서 해당 단어의 TF-IDF 점수를 가져오기
            tf_idf_scores = dtm[:, index].toarray().flatten() # 해당 단어가 있는 모든 문서에 대해 가져옴
            similar_words_tf_idf[word] = tf_idf_scores
    return similar_words_tf_idf

# 유사 단어 및 TF-IDF 값 출력 함수 정의
def print_similar_words_tf_idf(similar_words_tf_idf):
    for word, scores in similar_words_tf_idf.items():
        print(f"단어: {word}")
        # 0이 아닌 TF-IDF 점수와 해당 문서 인덱스 출력
        non_zero_scores = [(idx, score) for idx, score in enumerate(scores) if score > 0]
        if non_zero_scores:  # 0이 아닌 점수가 있는 경우에만 출력
            print("TF-IDF 점수:")
            for idx, score in non_zero_scores:
                print(f"  문서 {idx + 2}: {score}")
        else:
            print("해당 단어의 TF-IDF 점수가 모든 문서에서 0입니다.")
        print("\n")

In [16]:
def main(train_model=True):
    # 한국어 불용어 리스트 정의
    stop_words = set([
        '의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를',
        '으로', '자', '에', '와', '한', '하다', '에서', '에게', '이다', '위해'
    ])

    # 한국어 형태소 분석기 Okt 객체 생성
    okt = Okt()

    # 측정 시작
    start_time = time.time()

    # CSV 파일 읽어서 저장 기사 Content
    csv_file_path = '/content/drive/MyDrive/result/joongang_news(test).csv'
    titles = read_csv_file(csv_file_path)

    # 전처리된 제목 리스트 생성
    processed_titles = preprocess_titles(titles, okt, stop_words)

    # 모델 경로 설정
    model_path = '/content/drive/MyDrive/models/word2vec.model'

    if train_model:
        # 생성된 제목 리스트로 Word2Vec 모델 학습 및 저장
        word2vec_model = train_and_save_word2vec_model(processed_titles, model_path, seed=SEED)
    else:
        # 저장된 Word2Vec 모델 불러오기
        word2vec_model = load_word2vec_model(model_path)

    # TF-IDF 벡터화 및 밀집 행렬 생성, 가공처리해주어야함
    processed_titles_str = [' '.join(tokens) for tokens in processed_titles]
    feature_names, dtm = calculate_tfidf(processed_titles_str)

    # TF-IDF 결과 출력 (예제: 첫 10개 문서)
    print_tfidf_results(feature_names, dtm, num_docs=10)

    while True:
      # 사용자로부터 단어 입력 받기
      input_word = input("단어를 입력하세요 (exit을 입력하면 종료됩니다): ")

      # exit을 입력하면 루프 종료
      if input_word == 'exit':
          break

      # 유사한 단어 찾기
      similar_words = find_similar_words(word2vec_model, input_word, feature_names, dtm)

      # 유사 단어 출력
      print_similar_words_tf_idf(similar_words)

    end_time = time.time();
    print(f"총 걸린 시간: {end_time - start_time} 초")

if __name__ == "__main__":
    #main(train_model=True)  # 처음 실행 시 모델 학습 및 저장
    main(train_model=False)  # 이후 실행 시 저장된 모델 불러오기

Processing titles: 100%|██████████| 1234/1234 [01:19<00:00, 15.47it/s]


문서 1의 TF-IDF 결과:
  년생: 0.9013223519828854
  길방: 0.1884784087804431
  재물: 0.18053322627157262
  사랑: 0.13098078181747683
  건강: 0.12750963657523107
  보통: 0.12608388327494363
  수도: 0.07652333865269711
  무난: 0.060177742090524206
  양호: 0.05741785657873831
  행복: 0.05425858876752951
  나이: 0.051808070273493136
  말고: 0.051099320956320714
  베풂: 0.04711960219511078
  생길: 0.039424302291114106
  주기: 0.037825164982483096
  미운: 0.03389487389644813
  한마음: 0.030088871045262103
  기쁨: 0.029022779506174764
  집안: 0.02786250209968047
  사람: 0.027760232554574795
  마음: 0.027717269269883025
  자랑: 0.027607065278887823
  나무: 0.025549660478160357
  지출: 0.025549660478160357
  보다는: 0.02332329153257872
  오늘: 0.022007271517690708
  사는: 0.021917986600236715
  소식: 0.0214107738038027
  도움: 0.02109692258699709
  하기: 0.02045940059024967
  하면: 0.020371623359904823
  주의: 0.019933014832311698
  노력: 0.018201268666504677
  과음: 0.01737065004306841
  식복: 0.01737065004306841
  버리긴: 0.01737065004306841
  사공: 0.01737065004306841
  겹칠

KeyboardInterrupt: Interrupted by user

In [ ]:
# csv 기사내용 확인용
arr = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05557454882925253, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2068688369780905, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07918894015744724, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
print(len(arr))

non_zero_indices = [index for index, value in enumerate(arr) if value != 0]
print(non_zero_indices)


1234
[917, 926, 1154]
